In [1]:
import json
import sys
sys.path.insert(0, '../../')
from common.utils import id_from_uri
import common.metrics as metrics
from main import Title2Playlist

with open('../../data/playlist/data/mpd.slice.0-999.json') as fp:
    slice_json = json.load(fp)

playlist = slice_json['playlists'][1]
model = Title2Playlist('../../data/song/playlist_song.db')


/home/evan/ece5424/proj/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly id

In [6]:
def eval_r_precision_playlist(t2p_model: Title2Playlist, playlist_json):
    real_tuples = [(id_from_uri(track['track_uri']), id_from_uri(track['artist_uri'])) for track in playlist_json['tracks']]
    experimental_ids = t2p_model.create_playlist(playlist_json['name'], len(real_tuples))

    # SELECT artistID FROM songs WHERE songID IN (?,?,...,?,?)
    artist_query = "SELECT artistID FROM songs WHERE songID=? LIMIT 1"
    experimental_artists = [t2p_model.song_conn.execute(artist_query, [id]).fetchone() for id in experimental_ids]

    experimental_tuples = list(zip(experimental_ids, experimental_artists))
    return metrics.r_precision(real_tuples, experimental_tuples)

In [8]:
from tqdm import tqdm

total = 0
count = 0
for playlist in tqdm(slice_json['playlists']):
    new_score = eval_r_precision_playlist(model, playlist)
    count += 1
    total += new_score
    if count % 20 == 0:
        print(f"count: {count}; score: {new_score}; average score: {total/count}") 

print(f"FINAL AVERAGE R-PRECISION: {total / count}")

  2%|▏         | 21/1000 [00:26<13:02,  1.25it/s] 

count: 20; score: 0.0; average score: 0.08063447317189662


  4%|▍         | 40/1000 [00:31<03:48,  4.19it/s]

count: 40; score: 0.20689655172413793; average score: 0.07919725370340037


  6%|▌         | 60/1000 [00:39<06:17,  2.49it/s]

...

count: 960; score: 0.0; average score: 0.08221321710538274


 98%|█████████▊| 980/1000 [07:42<00:31,  1.57s/it]

count: 980; score: 0.00904977375565611; average score: 0.08207051700029581


100%|██████████| 1000/1000 [07:51<00:00,  2.12it/s]

count: 1000; score: 0.022727272727272728; average score: 0.0824959027276658
FINAL AVERAGE R-PRECISION: 0.0824959027276658


In [11]:
def eval_ndcg_playlist(t2p_model: Title2Playlist, playlist_json):
    real_ids = [id_from_uri(track['track_uri']) for track in playlist_json['tracks']]
    experimental_ids = t2p_model.create_playlist(playlist_json['name'], 500)

    return metrics.ndcg(real_ids, experimental_ids)

In [12]:
total = 0
count = 0
for playlist in tqdm(slice_json['playlists']):
    new_score = eval_ndcg_playlist(model, playlist)
    count += 1
    total += new_score
    if count % 20 == 0:
        print(f"count: {count}; score: {new_score}; average score: {total/count}") 

print(f"FINAL AVERAGE NDCG: {total / count}")

  2%|▏         | 21/1000 [00:03<02:53,  5.65it/s]

count: 20; score: 0.007489093171063151; average score: 0.1974232300679793


  4%|▍         | 41/1000 [00:07<02:55,  5.47it/s]

count: 40; score: 0.4990650125549674; average score: 0.19554248263496218


  6%|▌         | 61/1000 [00:11<02:49,  5.53it/s]

...

count: 960; score: 0.0485095024292715; average score: 0.18367967921805364


 98%|█████████▊| 981/1000 [02:58<00:03,  5.57it/s]

count: 980; score: 0.016060567289660627; average score: 0.183081755647454


100%|██████████| 1000/1000 [03:01<00:00,  5.50it/s]

count: 1000; score: 0.06427319603316908; average score: 0.18371941459581442
FINAL AVERAGE NDCG: 0.18371941459581442
